<h1 style="color:black;">Credit rating assignment</h1>
<p></p>

In [98]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.metrics import average_precision_score,roc_auc_score,make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [99]:
df = pd.read_csv("loan_data_small.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,int_rate,grade,home_ownership,annual_inc,loan_status,loan_amnt,purpose
0,1131156,1131156,15.61,D,OWN,20000.0,Fully Paid,9200,debt_consolidation
1,1526956,1526956,12.62,C,OWN,35000.0,Charged Off,6000,car
2,150283,150283,14.47,C,RENT,75000.0,Current,11000,other
3,1480461,1480461,11.99,B,RENT,45000.0,Current,8500,other
4,2188054,2188054,21.45,D,MORTGAGE,55000.0,Charged Off,20000,debt_consolidation


In [100]:
df.shape

(299999, 9)

In [101]:
df.dtypes

Unnamed: 0.1        int64
Unnamed: 0          int64
int_rate          float64
grade              object
home_ownership     object
annual_inc        float64
loan_status        object
loan_amnt           int64
purpose            object
dtype: object

In [102]:
df.isnull().sum()

Unnamed: 0.1      0
Unnamed: 0        0
int_rate          0
grade             0
home_ownership    0
annual_inc        0
loan_status       0
loan_amnt         0
purpose           0
dtype: int64

In [103]:
y = df["loan_status"].isin(["Charged Off","Default", "Does not meet the credit policy"]) 
y

0         False
1          True
2         False
3         False
4          True
          ...  
299994    False
299995    False
299996    False
299997    False
299998    False
Name: loan_status, Length: 299999, dtype: bool

In [104]:
df = df.drop(["Unnamed: 0", "loan_status"], axis=1)
df

,Unnamed: 0.1,int_rate,grade,home_ownership,annual_inc,loan_amnt,purpose
0,1131156,15.61,D,OWN,20000.0,9200,debt_consolidation
1,1526956,12.62,C,OWN,35000.0,6000,car
2,150283,14.47,C,RENT,75000.0,11000,other
3,1480461,11.99,B,RENT,45000.0,8500,other
4,2188054,21.45,D,MORTGAGE,55000.0,20000,debt_consolidation
...,...,...,...,...,...,...,...
299994,161327,11.55,B,RENT,60000.0,20000,debt_consolidation
299995,79980,11.06,B,MORTGAGE,135000.0,10000,home_improvement
299996,608153,8.39,B,MORTGAGE,70000.0,15000,debt_consolidation
299997,1176942,18.25,E,MORTGAGE,42000.0,21000,debt_consolidation


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299999 entries, 0 to 299998
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0.1    299999 non-null  int64  
 1   int_rate        299999 non-null  float64
 2   grade           299999 non-null  object 
 3   home_ownership  299999 non-null  object 
 4   annual_inc      299999 non-null  float64
 5   loan_amnt       299999 non-null  int64  
 6   purpose         299999 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 16.0+ MB


In [96]:
label_encoder = LabelEncoder()
df["grade"] = label_encoder.fit_transform(df["grade"])
df["purpose"] = label_encoder.fit_transform(df["purpose"])
df["home_ownership"] = label_encoder.fit_transform(df["home_ownership"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299999 entries, 0 to 299998
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0.1    299999 non-null  int64  
 1   int_rate        299999 non-null  float64
 2   grade           299999 non-null  int64  
 3   home_ownership  299999 non-null  int64  
 4   annual_inc      299999 non-null  float64
 5   loan_amnt       299999 non-null  int64  
 6   purpose         299999 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 16.0 MB


In [45]:
df

,Unnamed: 0.1,int_rate,grade,home_ownership,annual_inc,loan_amnt,purpose
0,1131156,15.61,3,4,20000.0,9200,2
1,1526956,12.62,2,4,35000.0,6000,0
2,150283,14.47,2,5,75000.0,11000,9
3,1480461,11.99,1,5,45000.0,8500,9
4,2188054,21.45,3,1,55000.0,20000,2
...,...,...,...,...,...,...,...
299994,161327,11.55,1,5,60000.0,20000,2
299995,79980,11.06,1,1,135000.0,10000,4
299996,608153,8.39,1,1,70000.0,15000,2
299997,1176942,18.25,4,1,42000.0,21000,2


In [47]:
preprocess = make_column_transformer(
    (StandardScaler(),['int_rate', 'annual_inc'], ),
    (OneHotEncoder(categories="auto",drop="first"),['grade', 'home_ownership','purpose'], )
)

In [48]:
X = preprocess.fit_transform(df)
X.shape

(299999, 26)

In [59]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(239999, 26) (239999,)
(60000, 26) (60000,)


<h3 style="color:green;">The models</h3>
<p></p>
<li><b>Model 1 SGD Classifier</b>: Vanilla version with max_iter set to 1000</li>
<li><b>Model 2 SGD Classifier round 2</b>: SGD Classifier with positive cases assigned a higher weight. One issue with our data is that positive cases are vastly outnumbered by negative cases (in other words, a model that says all cases are negative will have a pretty good accuracy). By overweighting positive cases in our model, we increase the efficacy of the model in finding an actual good solution</li>
<li><b>Model 3 SGD Classifier round 3</b>: Best SGD Classifier model after grid search</li>
<li><b>Model 4 Random Forest Classifier round 1</b>: Random Forest Classifier with base parameters (see below)</li>
<li><b>Model 5 Random Forest Classifier round 2</b>: Best model from grid search</li>
<li><b>Model 6 Gradient Booster Classfier</b></li>
<li><b>Model 7 Gradient Booster Classifier (2nd model)</b></li>

<h3 style="color:black;">Model_1</h3>
<p></p>

In [62]:
model_1 = SGDClassifier(random_state=42, max_iter=1000, loss="log_loss") #package version causes the different results
model_1.fit(x_train,y_train) #change if you used different variable names

print(model_1.score(x_train,y_train))
print(model_1.score(x_test,y_test))

0.8845495189563289
0.8857166666666667


In [64]:
y_test_pred = model_1.predict(x_test)
cfm_1 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_1.ravel()

y_train_pred = model_1.predict(x_train)
cfm_train_1 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_1.ravel()

accuracy_training_1 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_1 = (tp+tn)/(tp+tn+fp+fn)
precision_1  = precision_score(y_test, y_test_pred)
recall_1  = recall_score(y_test, y_test_pred)
f1_1  = f1_score(y_test, y_test_pred)
auc_1  = roc_auc_score(y_test, y_test_pred)
ap_1  = average_precision_score(y_test, y_test_pred)

print("Confusion Matrix: \n",cfm_1)
print("Training accuracy: ",accuracy_training_1)
print("Testing  accuracy: ",accuracy_testing_1)
print("Precision: ",precision_1)
print("Recall: ",recall_1)
print("F1-Score: ",f1_1)
print("AUC: ",auc_1)
print("Average Precision: ",ap_1)


Confusion Matrix: 
 [[53142     4]
 [ 6853     1]]
Training accuracy:  0.8845495189563289
Testing  accuracy:  0.8857166666666667
Precision:  0.2
Recall:  0.00014590020426028595
F1-Score:  0.000291587694999271
AUC:  0.5000353179190873
Average Precision:  0.11424584670751874


<h3 style="color:black;">Model_2</h3>
<p></p>

In [65]:
model_2 = SGDClassifier(random_state=42, max_iter=1000, loss="log_loss", class_weight={1:9}) 
model_2.fit(x_train,y_train) 

print(model_2.score(x_train,y_train))
print(model_2.score(x_test,y_test))


0.5556648152700636
0.55055


In [66]:
y_test_pred = model_2.predict(x_test)
cfm_2 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_2.ravel()

y_train_pred = model_2.predict(x_train)
cfm_train_2 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_2.ravel()

accuracy_training_2 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_2 = (tp+tn)/(tp+tn+fp+fn)
precision_2  = precision_score(y_test, y_test_pred)
recall_2  = recall_score(y_test, y_test_pred)
f1_2 = f1_score(y_test, y_test_pred)
auc_2 = roc_auc_score(y_test, y_test_pred)
ap_2 = average_precision_score(y_test, y_test_pred)

print("Confusion Matrix: \n",cfm_2)
print("Training accuracy: ",accuracy_training_2)
print("Testing  accuracy: ",accuracy_testing_2)
print("Precision: ",precision_2)
print("Recall: ",recall_2)
print("F1-Score: ",f1_2)
print("AUC: ",auc_2)
print("Average Precision: ",ap_2)

Confusion Matrix: 
 [[27990 25156]
 [ 1811  5043]]
Training accuracy:  0.5556648152700636
Testing  accuracy:  0.55055
Precision:  0.1669922845127322
Recall:  0.7357747300846221
F1-Score:  0.27220467978301355
AUC:  0.631218565885272
Average Precision:  0.1530520363969033


<h3 style="color:black;">Model_3</h3>
<p></p>

In [68]:
%%time
param_grid = {
    'alpha':(0.001, 0.01, 0.1),
    'l1_ratio': (0.01, 0.1, 1),
    'class_weight':({1:3},{1:6},{1:9})
}

#Do the search
gs_clf = GridSearchCV(SGDClassifier(random_state=42,
                                    max_iter=100,
                                    loss="log_loss",
                                    penalty="elasticnet"),
                      param_grid, cv=5, n_jobs=-1)
gs_clf.fit(x_train, y_train)

CPU times: total: 2.41 s
Wall time: 16.5 s


GridSearchCV(cv=5,
             estimator=SGDClassifier(loss='log_loss', max_iter=100,
                                     penalty='elasticnet', random_state=42),
             n_jobs=-1,
             param_grid={'alpha': (0.001, 0.01, 0.1),
                         'class_weight': ({1: 3}, {1: 6}, {1: 9}),
                         'l1_ratio': (0.01, 0.1, 1)})

In [69]:
gs_clf.best_params_

{'alpha': 0.1, 'class_weight': {1: 3}, 'l1_ratio': 1}

In [70]:
model_3 = SGDClassifier(random_state=42, 
                        max_iter=100, 
                        loss="log_loss", 
                        penalty="elasticnet",
                        alpha=0.1, 
                        class_weight={1:3}, 
                        l1_ratio=1) 
model_3.fit(x_train,y_train) 

print(model_3.score(x_train,y_train))
print(model_3.score(x_test,y_test))

0.8707244613519223
0.8702333333333333


In [71]:
y_test_pred = model_3.predict(x_test)
cfm_3 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_3.ravel()

y_train_pred = model_3.predict(x_train)
cfm_train_3 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_3.ravel()

accuracy_training_3 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_3 = (tp+tn)/(tp+tn+fp+fn)
precision_3  = precision_score(y_test, y_test_pred)
recall_3  = recall_score(y_test, y_test_pred)
f1_3  = f1_score(y_test, y_test_pred)
auc_3  = roc_auc_score(y_test, y_test_pred)
ap_3  = average_precision_score(y_test, y_test_pred)

print("Confusion Matrix: \n",cfm_3)
print("Training accuracy: ",accuracy_training_3)
print("Testing  accuracy: ",accuracy_testing_3)
print("Precision: ",precision_3)
print("Recall: ",recall_3)
print("F1-Score: ",f1_3)
print("AUC: ",auc_3)
print("Average Precision: ",ap_3)

Confusion Matrix: 
 [[51793  1353]
 [ 6433   421]]
Training accuracy:  0.8707244613519223
Testing  accuracy:  0.8702333333333333
Precision:  0.23731679819616686
Recall:  0.06142398599358039
F1-Score:  0.09758924432081595
AUC:  0.5179829070825163
Average Precision:  0.12179361035510938


<h3 style="color:black;">Model_4</h3>
<p></p>

In [73]:
model_4 = RandomForestClassifier(random_state=42,
                                 n_estimators=30,
                                 max_depth=6,
                                 min_samples_leaf=2000,
                                 min_samples_split=4000,
                                 class_weight={1:5})
model_4.fit(x_train,y_train)

RandomForestClassifier(class_weight={1: 5}, max_depth=6, min_samples_leaf=2000,
                       min_samples_split=4000, n_estimators=30,
                       random_state=42)

In [74]:
y_test_pred = model_4.predict(x_test)
cfm_4 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_4.ravel()

y_train_pred = model_4.predict(x_train)
cfm_train_4 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_4.ravel()

accuracy_training_4 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_4 = (tp+tn)/(tp+tn+fp+fn)
precision_4  = precision_score(y_test, y_test_pred)
recall_4  = recall_score(y_test, y_test_pred)
f1_4  = f1_score(y_test, y_test_pred)
auc_4  = roc_auc_score(y_test, y_test_pred)
ap_4  = average_precision_score(y_test, y_test_pred)

print("Confusion Matrix: \n",cfm_4)
print("Training accuracy: ",accuracy_training_4)
print("Testing  accuracy: ",accuracy_testing_4)
print("Precision: ",precision_4)
print("Recall: ",recall_4)
print("F1-Score: ",f1_4)
print("AUC: ",auc_4)
print("Average Precision: ",ap_4)

Confusion Matrix: 
 [[47744  5402]
 [ 5028  1826]]
Training accuracy:  0.8280534502227093
Testing  accuracy:  0.8261666666666667
Precision:  0.25262866629773106
Recall:  0.26641377297928215
F1-Score:  0.2593381621928703
AUC:  0.582384623290153
Average Precision:  0.15110375615110255


<h3 style="color:black;">Model_5</h3>
<p></p>

In [76]:
%%time
parameters = {
    'n_estimators': [800],
    'min_samples_split': [100],
    'class_weight': [{1:6}],
    'min_samples_leaf': [10] 
}
gs_clf = GridSearchCV(RandomForestClassifier(random_state=42),parameters,cv=5,n_jobs=-1,
                      scoring='f1')
gs_clf.fit(x_train, np.ravel(y_train))

CPU times: total: 12min 35s
Wall time: 31min 48s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': [{1: 6}], 'min_samples_leaf': [10],
                         'min_samples_split': [100], 'n_estimators': [800]},
             scoring='f1')

In [77]:
gs_clf.best_params_

{'class_weight': {1: 6},
 'min_samples_leaf': 10,
 'min_samples_split': 100,
 'n_estimators': 800}

In [78]:
model_5 = RandomForestClassifier(random_state=42,
                                 n_estimators=800,
                                 min_samples_leaf=10,
                                 min_samples_split=100,
                                 class_weight={1:6},
                                 n_jobs=-1)
model_5.fit(x_train,y_train)

RandomForestClassifier(class_weight={1: 6}, min_samples_leaf=10,
                       min_samples_split=100, n_estimators=800, n_jobs=-1,
                       random_state=42)

In [79]:
y_test_pred = model_5.predict(x_test)
cfm_5 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_5.ravel()

y_train_pred = model_5.predict(x_train)
cfm_train_5 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_5.ravel()

accuracy_training_5 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_5 = (tp+tn)/(tp+tn+fp+fn)
precision_5  = precision_score(y_test, y_test_pred)
recall_5  = recall_score(y_test, y_test_pred)
f1_5  = f1_score(y_test, y_test_pred)
auc_5  = roc_auc_score(y_test, y_test_pred)
ap_5  = average_precision_score(y_test, y_test_pred)

print("Training accuracy: ",accuracy_training_5)
print("Testing  accuracy: ",accuracy_testing_5)
print("confusion matrix:")
print(cfm_5)
print("precision: ",precision_5)
print("recall: ",recall_5)
print("f1 score: ",f1_5)
print("auc",auc_5)
print("ap",ap_5)

Training accuracy:  0.7789032454301893
Testing  accuracy:  0.7578666666666667
confusion matrix:
[[42079 11067]
 [ 3461  3393]]
precision:  0.23464730290456431
recall:  0.49503939305515027
f1 score:  0.318382283944825
auc 0.6434008540935255
ap 0.17384299174523685


<h3 style="color:black;">Model_6</h3>
<p></p>

In [82]:
sample_weight = np.array([4 if i == 1 else 1 for i in y_train])

model_6 = GradientBoostingClassifier(min_samples_split=100,
                                     max_depth=8,
                                     min_samples_leaf=100,
                                     n_estimators=400,
                                     subsample=0.6)

model_6.fit(x_train,y_train,sample_weight=sample_weight)

GradientBoostingClassifier(max_depth=8, min_samples_leaf=100,
                           min_samples_split=100, n_estimators=400,
                           subsample=0.6)

In [83]:
y_test_pred = model_6.predict(x_test)
cfm_6 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_6.ravel()

y_train_pred = model_6.predict(x_train)
cfm_train_6 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_6.ravel()

accuracy_training_6 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_6 = (tp+tn)/(tp+tn+fp+fn)
precision_6  = precision_score(y_test, y_test_pred)
recall_6  = recall_score(y_test, y_test_pred)
f1_6  = f1_score(y_test, y_test_pred)
auc_6  = roc_auc_score(y_test, y_test_pred)
ap_6  = average_precision_score(y_test, y_test_pred)

print("Training accuracy: ",accuracy_training_6)
print("Testing  accuracy: ",accuracy_testing_6)
print("confusion matrix:")
print(cfm_6)
print("precision: ",precision_6)
print("recall: ",recall_6)
print("f1 score: ",f1_6)
print("auc",auc_6)
print("ap",ap_6)

Training accuracy:  0.8213242555177314
Testing  accuracy:  0.8004333333333333
confusion matrix:
[[45278  7868]
 [ 4106  2748]]
precision:  0.258854559155991
recall:  0.4009337613072658
f1 score:  0.3145964510589582
auc 0.6264443766081732
ap 0.17221686536727893


<h3 style="color:black;">Model_7</h3>
<p></p>

In [84]:
sample_weight = np.array([5 if i == 1 else 1 for i in y_train])

model_7 = GradientBoostingClassifier(min_samples_split=100,
                                     max_depth=8,
                                     min_samples_leaf=100,
                                     n_estimators=400,
                                     subsample=0.6)
model_7.fit(x_train,y_train,sample_weight=sample_weight)

GradientBoostingClassifier(max_depth=8, min_samples_leaf=100,
                           min_samples_split=100, n_estimators=400,
                           subsample=0.6)

In [85]:
y_test_pred = model_7.predict(x_test)
cfm_7 = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cfm_7.ravel()

y_train_pred = model_7.predict(x_train)
cfm_train_7 = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = cfm_train_7.ravel()

accuracy_training_7 = (tp_train+tn_train)/(tp_train+tn_train+fp_train+fn_train)
accuracy_testing_7 = (tp+tn)/(tp+tn+fp+fn)
precision_7  = precision_score(y_test, y_test_pred)
recall_7  = recall_score(y_test, y_test_pred)
f1_7  = f1_score(y_test, y_test_pred)
auc_7  = roc_auc_score(y_test, y_test_pred)
ap_7  = average_precision_score(y_test, y_test_pred)

print("Training accuracy: ",accuracy_training_7)
print("Testing  accuracy: ",accuracy_testing_7)
print("confusion matrix:")
print(cfm_7)
print("precision: ",precision_7)
print("recall: ",recall_7)
print("f1 score: ",f1_7)
print("auc",auc_7)
print("ap",ap_7)

Training accuracy:  0.7775199063329431
Testing  accuracy:  0.7533
confusion matrix:
[[41625 11521]
 [ 3281  3573]]
precision:  0.23671657612296276
recall:  0.5213014298220018
f1 score:  0.325587752870421
auc 0.6522606197015778
ap 0.17808402292880254


<h3 style="color:black;">Results_df</h3>
<p></p>

In [89]:
results_df = pd.DataFrame(np.zeros(shape=(7,6)))
results_df.index=[1,2,3,4,5,6,7]
results_df.columns = ["accuracy","precision","recall","f1_score","AUC","AP"]
results_df.index.rename("Model",inplace=True)
results_df

,accuracy,precision,recall,f1_score,AUC,AP
Model,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
results_df.loc[[1,2,3,4,5,6,7], "accuracy"] = [accuracy_testing_1, accuracy_testing_2, accuracy_testing_3,accuracy_testing_4,accuracy_testing_5,accuracy_testing_6,accuracy_testing_7]
results_df.loc[[1,2,3,4,5,6,7], "precision"] = [precision_1, precision_2, precision_3,precision_4,precision_5,precision_6,precision_7]
results_df.loc[[1,2,3,4,5,6,7],"recall"] = [recall_1,recall_2,recall_3,recall_4,recall_5,recall_6,recall_7]
results_df.loc[[1,2,3,4,5,6,7],"f1_score"] = [f1_1,f1_2,f1_3,f1_4,f1_5,f1_6,f1_7]
results_df.loc[[1,2,3,4,5,6,7],"AUC"] = [auc_1,auc_2,auc_3,auc_4,auc_5,auc_6,auc_7]
results_df.loc[[1,2,3,4,5,6,7],"AP"] = [ap_1,ap_2,ap_3,ap_4,ap_5,ap_6,ap_7]
results_df

,accuracy,precision,recall,f1_score,AUC,AP
Model,,,,,,
1,0.885717,0.200000,0.000146,0.000292,0.500035,0.114246
2,0.550550,0.166992,0.735775,0.272205,0.631219,0.153052
3,0.870233,0.237317,0.061424,0.097589,0.517983,0.121794
4,0.826167,0.252629,0.266414,0.259338,0.582385,0.151104
5,0.757867,0.234647,0.495039,0.318382,0.643401,0.173843
6,0.800433,0.258855,0.400934,0.314596,0.626444,0.172217
7,0.753300,0.236717,0.521301,0.325588,0.652261,0.178084


In [92]:
pip install bokeh

   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
    --------------------------------------- 0.1/6.8 MB 1.1 MB/s eta 0:00:07
   - -------------------------------------- 0.3/6.8 MB 2.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/6.8 MB 3.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/6.8 MB 4.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.2/6.8 MB 4.4 MB/s eta 0:00:02
   ---------- ----------------------------- 1.7/6.8 MB 5.4 MB/s eta 0:00:01
   ----------- ---------------------------- 1.9/6.8 MB 5.7 MB/s eta 0:00:01
   ------------- -------------------------- 2.3/6.8 MB 5.4 MB/s eta 0:00:01
   --------------- ------------------------ 2.6/6.8 MB 5.8 MB/s eta 0:00:01
   ------------------ --------------------- 3.1/6.8 MB 5.9 MB/s eta 0:00:01
   -------------------- ------------------- 3.5/6.8 MB 6.4 MB/s eta 0:00:01
   ---------------------- 

In [93]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, LabelSet, HoverTool
output_notebook()

Loading BokehJS ...

In [94]:
#CHART 
models = ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5', 'Model 6', 'Model7']
accuracy = [accuracy_testing_1, accuracy_testing_2, accuracy_testing_3, accuracy_testing_4, accuracy_testing_5, accuracy_testing_6, accuracy_testing_7]
precision = [precision_1, precision_2, precision_3, precision_4, precision_5, precision_6, precision_7]
recall = [recall_1, recall_2, recall_3, recall_4, recall_5, recall_6, recall_7]
f1 = [f1_1, f1_2, f1_3, f1_4, f1_5, f1_6, f1_7]

data = {'Model':models,
       'Accuracy':accuracy,
       'Precision':precision,
       'Recall':recall,
       'F1score':f1}
source = ColumnDataSource(data=data)
tooltips = [
    ('Model', '@Model'),
    ('Accuracy', '@Accuracy{0.00}'),
    ('Precision', '@Precision{0.00}'),
    ('Recall', '@Recall{0.00}'),
    ('F1-score', '@F1score{0.00}'),
]
hover_tool = HoverTool(tooltips=tooltips)

accuracy_fig = figure(title='Accuracy', x_range=models, tools=[hover_tool])
accuracy_fig.vbar(x='Model', top='Accuracy', width=0.7, source=source)

precision_fig = figure(title='Precision', x_range=models, tools=[hover_tool])
precision_fig.vbar(x='Model', top='Precision', width=0.7, source=source)

recall_fig = figure(title='Recall', x_range=models, tools=[hover_tool])
recall_fig.vbar(x='Model', top='Recall', width=0.7, source=source)

f1_fig = figure(title='F1-score', x_range=models, tools=[hover_tool])
f1_fig.vbar(x='Model', top='F1score', width=0.7, source=source)

figures = [accuracy_fig, precision_fig, recall_fig, f1_fig]
grid = gridplot([ [fig] for fig in figures ])

show(grid)

In [95]:
#CHART

models = ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5', 'Model 6', 'Model7']
auc = [auc_1, auc_2, auc_3, auc_4, auc_5, auc_6, auc_7]
ap = [ap_1, ap_2, ap_3, ap_4, ap_5, ap_6, ap_7]

data = {'Model':models,
        'AUC':auc,
        'AP':ap}
source = ColumnDataSource(data=data)
tooltips = [
    ('Model', '@Model'),
    ('AUC', '@AUC{0.00}'),
    ('AP', '@AP{0.00}')
]
hover_tool = HoverTool(tooltips=tooltips)

auc_fig = figure(title='AUC', x_range=models, tools=[hover_tool])
auc_fig.vbar(x='Model', top='AUC', width=0.7, source=source)

ap_fig = figure(title='AP', x_range=models, tools=[hover_tool])
ap_fig.vbar(x='Model', top='AP', width=0.7, source=source)

figures = [auc_fig, ap_fig]
grid = gridplot([ [fig] for fig in figures ])


show(grid)